In [176]:
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [177]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url ='https://leandrolessa.com.br/coleta-de-dados-na-web/'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Localiza a tabela pelo ID
table = soup.find('table', {'id': 'tablepress-1'})

# Extrai os cabeçalhos da tabela
headers = [header.text.strip() for header in table.find_all('th')]

# Extrai as linhas da tabela
rows = table.find_all('tr')[1:]  # Ignora o cabeçalho
# Preenche diretamente em uma variável tipo lista de dicionário
data = []
for row in rows:
    values = [cell.text.strip() for cell in row.find_all('td')]
    data.append(dict(zip(headers, values)))

# Converte para JSON e depois para lista de dicionários
json_data = json.dumps(data)
dict_list = json.loads(json_data)

In [200]:
# Criação do DataFrame tb_classe_produto
tb_classe_produto = pd.DataFrame(dict_list)[['cod_classe_produto', 'classe_produto']].drop_duplicates()

# Renomeia as colunas para os nomes desejados
tb_classe_produto.rename(columns={'cod_classe_produto': 'cod_classe', 'classe_produto': 'classe'}, inplace=True)

# Define o campo 'cod_classe' como chave primária
tb_classe_produto.set_index('cod_classe', inplace=True)

# Exibe o DataFrame resultante
print(tb_classe_produto.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 1 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   classe  5 non-null      object
dtypes: object(1)
memory usage: 80.0+ bytes
None


In [179]:
# Criação do DataFrame tb_produto
tb_produto = pd.DataFrame(dict_list)[['cod_produto', 'nome_produto', 'cod_classe_produto']]

# Renomeia as colunas para os nomes desejados
tb_produto.rename(columns={'cod_produto': 'cod_produto', 'nome_produto': 'prod', 'cod_classe_produto': 'cod_classe'}, inplace=True)

# Define o campo 'cod_produto' como chave primária
tb_produto.set_index('cod_produto', inplace=True)

# Exibe o DataFrame resultante
print(tb_produto)

                       prod cod_classe
cod_produto                           
1000            Sabão em pó          1
1001              Amaciante          1
1002             Detergente          1
1003         Água sanitária          1
1004         Esponja de aço          1
...                     ...        ...
1082                  Peixe          5
1083                 Frango          5
1084         Carne vermelha          5
1085             Carne seca          5
1086               Salsicha          5

[87 rows x 2 columns]


In [181]:
# Criação do DataFrame df_estado_regiao com as colunas renomeadas
tb_estado = pd.read_csv('estado_regiao.csv', sep=';', encoding='utf-8')

# Renomeia as colunas
tb_estado.rename(columns={'id_estado': 'cod_estado'}, inplace=True)

# Define 'cod_estado' como chave primária
tb_estado.set_index('cod_estado', inplace=True)

# Exibe o DataFrame resultante
print(tb_estado)

           sigla               estado        regiao    pais
cod_estado                                                 
1             AC                 Acre         Norte  Brasil
2             AL              Alagoas      Nordeste  Brasil
3             AP                Amapá         Norte  Brasil
4             AM             Amazonas         Norte  Brasil
5             BA                Bahia      Nordeste  Brasil
6             CE                Ceará      Nordeste  Brasil
7             DF     Distrito Federal  Centro-Oeste  Brasil
8             ES       Espírito Santo       Sudeste  Brasil
9             GO                Goiás  Centro-Oeste  Brasil
10            MA             Maranhão      Nordeste  Brasil
11            MT          Mato Grosso  Centro-Oeste  Brasil
12            MS   Mato Grosso do Sul  Centro-Oeste  Brasil
13            MG         Minas Gerais       Sudeste  Brasil
14            PA                 Pará         Norte  Brasil
15            PB              Paraíba   

In [182]:
# Carrega o arquivo JSON em um DataFrame
with open('dados_clientes.json', encoding='latin1') as f:
    data = json.load(f)

# Criação do DataFrame tb_clientes
tb_clientes = pd.DataFrame(data['cliente'])[['cod_cliente', 'cod_estado', 'sexo', 'idade', 'qtd_filhos', 'hipertenso', 'diabetes']]

# Define o campo 'cod_cliente' como chave primária
tb_clientes.set_index('cod_cliente', inplace=True)

# Exibe o DataFrame resultante
print(tb_clientes.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 1 to 3000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cod_estado  3000 non-null   int64 
 1   sexo        3000 non-null   object
 2   idade       3000 non-null   int64 
 3   qtd_filhos  3000 non-null   int64 
 4   hipertenso  3000 non-null   int64 
 5   diabetes    3000 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 164.1+ KB
None


In [196]:
# Carrega o arquivo Excel em um DataFrame
df_compras = pd.read_excel('compras.xls')

# Verifica os dados ausentes na coluna 'cod_produto'
print(f"Dados ausentes em 'cod_produto': {df_compras['cod_produto'].isna().sum()}")

# Remove as linhas com valores nulos em 'cod_produto'
df_compras.dropna(subset=['cod_produto'], inplace=True)

# Cria o DataFrame tb_compra
tb_compra = df_compras

# Exibe informações sobre o DataFrame resultante
# Ajusta o tipo da coluna 'cod_produto' para int64
tb_compra['cod_produto'] = tb_compra['cod_produto'].astype('int64')

# Exibe informações sobre o DataFrame resultante
print(tb_compra.info())

Dados ausentes em 'cod_produto': 10
<class 'pandas.core.frame.DataFrame'>
Index: 2990 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cod_cliente         2990 non-null   int64  
 1   cod_produto         2990 non-null   int64  
 2   qtd_produto         2990 non-null   int64  
 3   valor_unitario      2985 non-null   float64
 4   valor_total_compra  2969 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 140.2 KB
None


In [212]:
# Faz a associação entre tb_produto e tb_classe_produto
tb_produto_classe = tb_produto.merge(tb_classe_produto, left_on='cod_classe', right_index=True, how='left')

# Exibe o DataFrame resultante
print(tb_produto_classe)

                       prod cod_classe               classe
cod_produto                                                
1000            Sabão em pó          1  Produtos de Limpeza
1001              Amaciante          1  Produtos de Limpeza
1002             Detergente          1  Produtos de Limpeza
1003         Água sanitária          1  Produtos de Limpeza
1004         Esponja de aço          1  Produtos de Limpeza
...                     ...        ...                  ...
1082                  Peixe          5       Carnes e Frios
1083                 Frango          5       Carnes e Frios
1084         Carne vermelha          5       Carnes e Frios
1085             Carne seca          5       Carnes e Frios
1086               Salsicha          5       Carnes e Frios

[87 rows x 3 columns]


In [211]:
# Faz a associação entre tb_clientes e tb_estado
tb_clientes_estado = tb_clientes.merge(tb_estado[['sigla']], left_on='cod_estado', right_index=True, how='left')

# Exibe o DataFrame resultante
# Exibe todos os campos do DataFrame resultante
print(tb_clientes_estado[['sigla', 'sexo', 'idade', 'qtd_filhos', 'hipertenso', 'diabetes']])

            sigla       sexo  idade  qtd_filhos  hipertenso  diabetes
cod_cliente                                                          
1              MG   feminino     26           2           0         1
2              AL   feminino     58           0           0         0
3              MT   feminino     63           1           1         1
4              MT   feminino     58           2           1         1
5              BA  masculino     70           0           0         1
...           ...        ...    ...         ...         ...       ...
2996           MS   feminino     63           1           0         1
2997           DF   feminino     35           0           0         0
2998           MT   feminino     35           2           0         0
2999           RN   feminino     20           0           1         0
3000           MT  masculino     73           0           1         1

[3000 rows x 6 columns]


In [215]:
# Faz a associação entre tb_compra e tb_clientes_estado
tb_compra_clientes = tb_compra.merge(tb_clientes_estado, left_on='cod_cliente', right_index=True, how='left')

# Exibe o DataFrame resultante
# Faz a associação para tb_produto_classe pelo cod_produto
tb_compra_clientes_produto = tb_compra_clientes.merge(tb_produto_classe, left_on='cod_produto', right_index=True, how='left')

# Exibe o DataFrame resultante
print(tb_compra_clientes_produto.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2990 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cod_cliente         2990 non-null   int64  
 1   cod_produto         2990 non-null   object 
 2   qtd_produto         2990 non-null   int64  
 3   valor_unitario      2985 non-null   float64
 4   valor_total_compra  2969 non-null   float64
 5   cod_estado          2990 non-null   int64  
 6   sexo                2990 non-null   object 
 7   idade               2990 non-null   int64  
 8   qtd_filhos          2990 non-null   int64  
 9   hipertenso          2990 non-null   int64  
 10  diabetes            2990 non-null   int64  
 11  sigla               2990 non-null   object 
 12  prod                2990 non-null   object 
 13  cod_classe          2990 non-null   object 
 14  classe              2990 non-null   object 
dtypes: float64(2), int64(7), object(6)
memory usage: 373.8+ KB
N

In [216]:
# Filtra os residentes de Minas Gerais
mg_data = tb_compra_clientes_produto[tb_compra_clientes_produto['sigla'] == 'MG']

# Calcula a média do valor total das compras por categoria (classe)
mean_values_by_class = mg_data.groupby('classe')['valor_total_compra'].mean()

# Identifica a categoria com a maior média
highest_avg_category = mean_values_by_class.idxmax()
highest_avg_value = mean_values_by_class.max()

print(f"A categoria com a maior média de valor total em compras para residentes de MG é '{highest_avg_category}' com uma média de {highest_avg_value:.2f}.")

A categoria com a maior média de valor total em compras para residentes de MG é 'Carnes e Frios' com uma média de 78.43.


In [217]:
# Filtra os consumidores diagnosticados com diabetes
diabetes_data = tb_compra_clientes_produto[tb_compra_clientes_produto['diabetes'] == 1]

# Agrupa os dados pelo produto e soma a quantidade de produtos vendidos
sales_by_product = diabetes_data.groupby('prod')['qtd_produto'].sum()

# Identifica o produto com o maior número de vendas
most_sold_product = sales_by_product.idxmax()
most_sold_quantity = sales_by_product.max()

print(f"O produto com o maior número de vendas entre consumidores diagnosticados com diabetes é '{most_sold_product}' com {most_sold_quantity} unidades vendidas.")

O produto com o maior número de vendas entre consumidores diagnosticados com diabetes é 'Achocolatado' com 78 unidades vendidas.


In [218]:
# Filtra os grupos com base nas condições fornecidas
grupo_1 = tb_compra_clientes_produto[(tb_compra_clientes_produto['sexo'] == 'masculino') &
                                     (tb_compra_clientes_produto['hipertenso'] == 1) &
                                     (tb_compra_clientes_produto['diabetes'] == 0)]

grupo_2 = tb_compra_clientes_produto[(tb_compra_clientes_produto['sexo'] == 'feminino') &
                                     (tb_compra_clientes_produto['hipertenso'] == 1) &
                                     (tb_compra_clientes_produto['diabetes'] == 0)]

grupo_3 = tb_compra_clientes_produto[(tb_compra_clientes_produto['sexo'] == 'masculino') &
                                     (tb_compra_clientes_produto['hipertenso'] == 0) &
                                     (tb_compra_clientes_produto['diabetes'] == 0)]

grupo_4 = tb_compra_clientes_produto[(tb_compra_clientes_produto['sexo'] == 'masculino') &
                                     (tb_compra_clientes_produto['hipertenso'] == 1) &
                                     (tb_compra_clientes_produto['diabetes'] == 1)]

# Calcula a média de idade para cada grupo
media_idade_grupo_1 = grupo_1['idade'].mean()
media_idade_grupo_2 = grupo_2['idade'].mean()
media_idade_grupo_3 = grupo_3['idade'].mean()
media_idade_grupo_4 = grupo_4['idade'].mean()

# Identifica o grupo com a maior média de idade
medias = {
    "Homem com histórico de hipertensão, mas sem histórico de diabetes.": media_idade_grupo_1,
    "Mulheres com histórico de hipertensão, mas sem histórico de diabetes.": media_idade_grupo_2,
    "Homens sem histórico de hipertensão ou diabetes.": media_idade_grupo_3,
    "Homem com histórico de hipertensão e diabetes.": media_idade_grupo_4
}

grupo_maior_media = max(medias, key=medias.get)
maior_media = medias[grupo_maior_media]

print(f"O grupo com a maior média de idade é '{grupo_maior_media}' com uma média de {maior_media:.2f} anos.")

O grupo com a maior média de idade é 'Homens sem histórico de hipertensão ou diabetes.' com uma média de 47.06 anos.


In [226]:
# Agrupa os dados de tb_compra_clientes_produto_regiao por região e soma o valor total das compras
valor_total_por_regiao = tb_compra_clientes_produto_regiao.groupby('regiao')['valor_total_compra'].sum()

# Identifica a região com o maior valor total de compras
regiao_maior_valor = valor_total_por_regiao.idxmax()
maior_valor = valor_total_por_regiao.max()

print(f"A região com o maior valor total de compras é '{regiao_maior_valor}' com um total de {maior_valor:.2f}.")

A região com o maior valor total de compras é 'Nordeste' com um total de 21528.46.


In [220]:
# Filtra os consumidores diagnosticados com hipertensão e diabetes
hipertensao_diabetes_data = tb_compra_clientes_produto[(tb_compra_clientes_produto['hipertenso'] == 1) &
                                                       (tb_compra_clientes_produto['diabetes'] == 1)]

# Agrupa os dados pela classe de produto e soma a quantidade de produtos comprados
compras_por_classe = hipertensao_diabetes_data.groupby('classe')['qtd_produto'].sum()

# Identifica a classe com o menor número de compras
classe_menor_compras = compras_por_classe.idxmin()
menor_compras = compras_por_classe.min()

print(f"A classe de produto com o menor número de compras entre pessoas diagnosticadas com hipertensão e diabetes é '{classe_menor_compras}' com {menor_compras} compras.")

A classe de produto com o menor número de compras entre pessoas diagnosticadas com hipertensão e diabetes é 'Bebidas' com 115 compras.


In [229]:
# Filtra os dados para a região Sudeste
sudeste_data = tb_compra_clientes_produto_regiao[tb_compra_clientes_produto_regiao['regiao'] == 'Sudeste']

# Agrupa os dados pelo produto e soma a quantidade de produtos comprados
compras_por_produto = sudeste_data.groupby('prod')['qtd_produto'].sum()

# Identifica o produto com o maior número de compras
produto_mais_comprado = compras_por_produto.idxmax()
quantidade_mais_comprada = compras_por_produto.max()
# Filtra os itens específicos e calcula a contagem
itens_especificos = ['Fubá', 'Presunto', 'Bolacha', 'Sardinha']
contagem_itens = compras_por_produto[compras_por_produto.index.isin(itens_especificos)]

# Exibe a contagem dos itens
print(contagem_itens)
print(f"O produto com o maior número de compras para os residentes da região Sudeste é '{produto_mais_comprado}' com {quantidade_mais_comprada} unidades compradas.")

prod
Bolacha      5
Fubá        25
Presunto    18
Sardinha    25
Name: qtd_produto, dtype: int64
O produto com o maior número de compras para os residentes da região Sudeste é 'Óleo de cozinha' com 28 unidades compradas.


In [ ]:
db.createCollection('forum_debates')
